In [2]:
# params for this protein 

params = { 
    'handle': 'bgl3',
    'nucleophile': 383,
    'acid_base': 178, 
    'backup': 309, 
    'ligand': 480, 
}

In [3]:
# set up the input files 

In [4]:
# puzzle setup 

puz_setup = 'version: 1\n{{\n"backbone_locked": "{}||"\n}}'
puz_fn = params['handle'] + '.puzzle_setup'
with open( puz_fn, 'w' ) as fn:
    fn.write( puz_setup.format( params[ 'ligand' ] ) ) 

In [5]:
! cat bgl3.puzzle_setup

version: 1
{
"backbone_locked": "480||"
}

In [6]:
# constraint file 

cst_template = '''
# chemical restraints for pNPG hydrolysis by {handle}

# pNPG (residue {ligand}) and acid/base E{acid_base}
AtomPair O2 {ligand} OE2 {acid_base} HARMONIC 1.8 0.18
Angle O2 {ligand} OE2 {acid_base} CD {acid_base} HARMONIC 0 10
Dihedral O2 {ligand} OE2 {acid_base} CD {acid_base} CG {acid_base} HARMONIC 0 10

# pNPG and nucleophile E{nucleophile}
AtomPair C5 {ligand} OE2 {nucleophile} HARMONIC 2.0 0.20
Angle O2 {ligand} C5 {ligand} OE2 {nucleophile} HARMONIC 0 10
Angle C5 {ligand} OE2 {nucleophile} CD {nucleophile} HARMONIC 0 10
Dihedral C5 {ligand} OE2 {nucleophile} CD {nucleophile} CG {nucleophile} HARMONIC 0 10

# nucleophile E{nucleophile} and Y{backup}
AtomPair OE2 {nucleophile} OH {backup} HARMONIC 3.0 0.30
'''

with open( '{}_pNPG.cnstr'.format( params[ 'handle' ] ), 'w' ) as fn:
    fn.write( cst_template.format( **params ) )

In [7]:
! cat bgl3_pNPG.cnstr


# chemical restraints for pNPG hydrolysis by bgl3

# pNPG (residue 480) and acid/base E178
AtomPair O2 480 OE2 178 HARMONIC 1.8 0.18
Angle O2 480 OE2 178 CD 178 HARMONIC 0 10
Dihedral O2 480 OE2 178 CD 178 CG 178 HARMONIC 0 10

# pNPG and nucleophile E383
AtomPair C5 480 OE2 383 HARMONIC 2.0 0.20
Angle O2 480 C5 480 OE2 383 HARMONIC 0 10
Angle C5 480 OE2 383 CD 383 HARMONIC 0 10
Dihedral C5 480 OE2 383 CD 383 CG 383 HARMONIC 0 10

# nucleophile E383 and Y309
AtomPair OE2 383 OH 309 HARMONIC 3.0 0.30


In [8]:
# enzyme design header for input PDB 

enzdes_header='''
REMARK   0 NE TEMPLATE X LG1 1 MATCH MOTIF A GLU {nucleophile} 1
REMARK   0 NE TEMPLATE X LG1 1 MATCH MOTIF A GLU {acid_base} 2
REMARK   0 NE TEMPLATE A GLU {nucleophile} MATCH MOTIF A TYR {backup} 3
'''

with open( 'enzdes_header.txt', 'w' ) as fn:
    fn.write( enzdes_header.format( **params ) ) 

In [9]:
! cat enzdes_header.txt


REMARK   0 NE TEMPLATE X LG1 1 MATCH MOTIF A GLU 383 1
REMARK   0 NE TEMPLATE X LG1 1 MATCH MOTIF A GLU 178 2
REMARK   0 NE TEMPLATE A GLU 383 MATCH MOTIF A TYR 309 3


In [24]:
# make a PDB file suitable for loading into enzyme design 

! ( cat enzdes_header.txt && grep '^ATOM' ../hybridize/S_0007.pdb && cat LG_0001.pdb ) > tss_input.pdb

In [25]:
! head tss_input.pdb
! echo '... (snip) ...'
! tail tss_input.pdb


REMARK   0 NE TEMPLATE X LG1 1 MATCH MOTIF A GLU 383 1
REMARK   0 NE TEMPLATE X LG1 1 MATCH MOTIF A GLU 178 2
REMARK   0 NE TEMPLATE A GLU 383 MATCH MOTIF A TYR 309 3
ATOM      1  N   MET A   1      25.444   8.852  25.251  1.00  0.00           N  
ATOM      2  CA  MET A   1      24.729   9.226  24.035  1.00  0.00           C  
ATOM      3  C   MET A   1      25.702   9.479  22.906  1.00  0.00           C  
ATOM      4  O   MET A   1      26.626  10.281  23.050  1.00  0.00           O  
ATOM      5  CB  MET A   1      23.880  10.477  24.274  1.00  0.00           C  
ATOM      6  CG  MET A   1      22.819  10.336  25.361  1.00  0.00           C  
... (snip) ...
HETATM   33  O7  LG1 X   1      15.895  16.449  -3.151  1.00 20.00           O  
HETATM   34  H14 LG1 X   1      19.166  13.549  -1.684  1.00 20.00           H  
HETATM   35  C14 LG1 X   1      18.487  16.260  -2.000  1.00 20.00           C  
HETATM   36  H15 LG1 X   1      20.242  17.175  -1.198  1.00 20.00           H  
HETATM 

In [26]:
! open tss_input.pdb